In [5]:
!pip install kagglehub --upgrade
!pip install -Iv tensorflow==2.13.0
!pip install pandas
!pip install seaborn

Using pip 24.3.1 from C:\Users\chase_onrf9ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip (python 3.11)

ERROR: Could not install packages due to an OSError.
Check the permissions.
Traceback (most recent call last):
  File "C:\Users\chase_onrf9ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_internal\commands\install.py", line 457, in run
    installed = install_given_reqs(
                ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\chase_onrf9ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_internal\req\__init__.py", line 70, in install_given_reqs
    requirement.install(
  File "C:\Users\chase_onrf9ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_internal\req\req_install.py", line 867, in install
    install_wheel(
  File "C:\Users\chase_onrf9ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Pytho


  Obtaining dependency information for tensorflow==2.13.0 from https://files.pythonhosted.org/packages/9e/b8/ed5f794359d05cd0bffb894c6418da87b93016ee17b669d55c45d1bd5d5b/tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/2f/2f/3c84f675931ce3bcbc7e23acbba1e5d7f05ce769adab48322de57a9f5928/tensorflow_intel-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.13.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13d

In [6]:
import os
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
from collections import defaultdict

In [7]:
path = kagglehub.dataset_download("shuyangli94/food-com-recipes-and-user-interactions")
print("Path to dataset files:", path)

# List all files in the dataset directory
dataset_files = os.listdir(path)
print("Files in the dataset:", dataset_files)

100%|██████████| 267M/267M [00:05<00:00, 48.5MB/s] 

Extracting files...


Path to dataset files: C:\Users\chase_onrf9ju\.cache\kagglehub\datasets\shuyangli94\food-com-recipes-and-user-interactions\versions\2
Files in the dataset: ['ingr_map.pkl', 'interactions_test.csv', 'interactions_train.csv', 'interactions_validation.csv', 'PP_recipes.csv', 'PP_users.csv', 'RAW_interactions.csv', 'RAW_recipes.csv']


In [8]:
# Dictionary to store DataFrames for each useful file
dataframes = {}

# Load each useful CSV file into a DataFrame dictionary
for file_name in dataset_files:
    file_path = os.path.join(path, file_name)
    if file_name.endswith('.csv') and os.path.exists(file_path):
        print(f"Loading {file_name} into a DataFrame...")
        df_name = file_name.split('.')[0]  # Use filename without extension as key
        dataframes[df_name] = pd.read_csv(file_path)
        print(f"Loaded {file_name} with {dataframes[df_name].shape[0]} rows and {dataframes[df_name].shape[1]} columns.")

Loading interactions_test.csv into a DataFrame...
Loaded interactions_test.csv with 12455 rows and 6 columns.
Loading interactions_train.csv into a DataFrame...
Loaded interactions_train.csv with 698901 rows and 6 columns.
Loading interactions_validation.csv into a DataFrame...
Loaded interactions_validation.csv with 7023 rows and 6 columns.
Loading PP_recipes.csv into a DataFrame...
Loaded PP_recipes.csv with 178265 rows and 8 columns.
Loading PP_users.csv into a DataFrame...
Loaded PP_users.csv with 25076 rows and 6 columns.
Loading RAW_interactions.csv into a DataFrame...
Loaded RAW_interactions.csv with 1132367 rows and 5 columns.
Loading RAW_recipes.csv into a DataFrame...
Loaded RAW_recipes.csv with 231637 rows and 12 columns.


In [9]:
raw_interactions_df = dataframes.get('RAW_interactions')

In [54]:
# Generate userIDs, recipeIDs, and put interactions into a set
userIDs = {}
recipeIDs = {}
interactions = []

for i in range(raw_interactions_df.shape[0]):
    row = raw_interactions_df.iloc[i]
    user = row['user_id']
    recipe = row['recipe_id']
    rating = row['rating']
    if not user in userIDs: userIDs[user] = len(userIDs)
    if not recipe in recipeIDs: recipeIDs[recipe] = len(recipeIDs)
    interactions.append((user,recipe,rating))

random.shuffle(interactions)

# Split interactions into training and testing sets
nTrain = int(len(interactions) * 0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [62]:
# Find the recipes per user, users per recipe, and global average rating
recipesPerUser = defaultdict(list)
usersPerRecipe = defaultdict(list)
ratingsPerUser = defaultdict(list)
ratingsPerRecipe = defaultdict(list)
for user,recipe,rating in interactionsTrain:
    recipesPerUser[user].append(recipe)
    usersPerRecipe[recipe].append(user)
    ratingsPerUser[user].append(rating)
    ratingsPerRecipe[recipe].append(rating)

globalAvg = sum([rating for _,_,rating in interactionsTrain]) / len(interactionsTrain)

In [58]:
# Baseline: Find MSE of predicting global average every time
MSE = 0
for user,recipe,rating in interactionsTest:
    error = (rating - globalAvg) ** 2
    MSE += error

MSE /= len(interactionsTest)

print("Test MSE (Using Global Averages) = " + str(MSE))

Test MSE (Using Global Averages) = 1.5848681885817457


In [64]:
# Baseline: Find MSE of predicting user average, or global average if there is none
MSE = 0
for user,recipe,rating in interactionsTest:
    if user in recipesPerUser:
        avgUserRating = sum(ratingsPerUser[user]) / len(ratingsPerUser[user])
        error = (rating - avgUserRating) ** 2
    else:
        error = (rating - globalAvg) ** 2
    MSE += error

MSE /= len(interactionsTest)

print("Test MSE (Using User Averages) = " + str(MSE))

Test MSE (Using User Averages) = 1.5632766663314015


In [65]:
# Baseline: Find MSE of predicting recipe average, or global average if there is none
MSE = 0
for user,recipe,rating in interactionsTest:
    if recipe in usersPerRecipe:
        avgRecipeRating = sum(ratingsPerRecipe[recipe]) / len(ratingsPerRecipe[recipe])
        error = (rating - avgRecipeRating) ** 2
    else:
        error = (rating - globalAvg) ** 2
    MSE += error

MSE /= len(interactionsTest)

print("Test MSE (Using Recipe Averages) = " + str(MSE))

Test MSE (Using Recipe Averages) = 1.7537650753993466


In [66]:
# Baseline: Find MSE of predicting recipe average or user average depending on total count, or global average if there is none
MSE = 0
for user,recipe,rating in interactionsTest:
    if recipe in usersPerRecipe and (user not in recipesPerUser or len(usersPerRecipe[recipe]) >= len(recipesPerUser[user])):
        avgRecipeRating = sum(ratingsPerRecipe[recipe]) / len(ratingsPerRecipe[recipe])
        error = (rating - avgRecipeRating) ** 2
    elif user in recipesPerUser and (recipe not in usersPerRecipe or len(usersPerRecipe[recipe]) <= len(recipesPerUser[user])):
        avgUserRating = sum(ratingsPerUser[user]) / len(ratingsPerUser[user])
        error = (rating - avgUserRating) ** 2
    else:
        error = (rating - globalAvg) ** 2
    MSE += error

MSE /= len(interactionsTest)

print("Test MSE (Using Either Average) = " + str(MSE))

Test MSE (Using Either Average) = 1.522573740651676


In [ ]:
# Latent Factor Model to be used for our predictive problem

class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.cast(tf.Variable(mu), dtype=tf.float32)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaR = tf.Variable(tf.random.normal([len(recipeIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaR = tf.Variable(tf.random.normal([len(recipeIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, r):
        prediction = self.alpha + self.betaU[u] + self.betaR[r] +\
            tf.tensordot(self.gammaU[u], self.gammaR[r], 1)
        return prediction

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaR**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaR**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleR):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        r = tf.convert_to_tensor(sampleR, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_r = tf.nn.embedding_lookup(self.betaR, r)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_r = tf.nn.embedding_lookup(self.gammaR, r)
        prediction = self.alpha + beta_u + beta_r + tf.reduce_sum(tf.multiply(gamma_u, gamma_r), 1)
        return prediction
    
    # Loss
    def call(self, sampleUser, sampleRecipe, sampleRating):
        prediction = self.predictSample(sampleUser, sampleRecipe)
        rating = tf.convert_to_tensor(sampleRating, dtype=tf.float32)
        return tf.nn.l2_loss(prediction - rating) / len(sampleRating)

In [ ]:
# Our optimizer and model

optimizer = tf.keras.optimizers.Adam(0.1)
modelLFM = LatentFactorModel(globalAvg, 12, 0.00001)

In [ ]:
# One iteration of our training step

def trainingStep(model, interactions):
    Nsamples = 250000
    with tf.GradientTape() as tape:
        sampleUser, sampleRecipe, sampleRating = [], [], []
        for _ in range(Nsamples):
            user,recipe,rating = random.choice(interactions)
            sampleUser.append(userIDs[user])
            sampleRecipe.append(recipeIDs[recipe])
            sampleRating.append(rating)

        loss = model(sampleUser,sampleRecipe,sampleRating)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [ ]:
# Iterate through 100 runs to fine tune our model

for i in range(100):
    obj = trainingStep(modelLFM, interactionsTrain)
    print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 1, objective = 0.80425376
iteration 2, objective = 0.7704608
iteration 3, objective = 0.7510032
iteration 4, objective = 0.7427501
iteration 5, objective = 0.74849945
iteration 6, objective = 0.7379982
iteration 7, objective = 0.7377526
iteration 8, objective = 0.74505186
iteration 9, objective = 0.73226947
iteration 10, objective = 0.7289218
iteration 11, objective = 0.7298265
iteration 12, objective = 0.7385655
iteration 13, objective = 0.73044527
iteration 14, objective = 0.7353144
iteration 15, objective = 0.73535186
iteration 16, objective = 0.73569643
iteration 17, objective = 0.7369458
iteration 18, objective = 0.73214436
iteration 19, objective = 0.72963095
iteration 20, objective = 0.73205996
iteration 21, objective = 0.73308593
iteration 22, objective = 0.734856
iteration 23, objective = 0.7324256
iteration 24, objective = 0.73736537
iteration 25, objective = 0.73690605
iteration 26, objective = 0.74310726
iteration 27, objective = 0.73716605
iteration 28, objective

In [ ]:
# Testing our model

MSE = 0

predictions = modelLFM.predictSample([userIDs[interaction[0]] for interaction in interactionsTest],[recipeIDs[interaction[1]] for interaction in interactionsTest])

for i in range(len(interactionsTest)):
    MSE += (interactionsTest[i][2] - predictions[i]) ** 2

MSE /= len(interactionsTest)
print("Test MSE = " + str(float(MSE)))

Test MSE = 1.4894706010818481
